In [1]:
import os
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드. .env에 선언된 것 가져옴
load_dotenv()

True

## Upstage

Upstage는 인간 수준 이상의 성능 LLM 구성 요소를 제공하는 데 특화된 선도적인 인공 지능(AI) 회사입니다.

Upstage 사의 Upstage Solar는 작지만 강력한 LLM(대형 언어 모델)입니다.

패키지를 설치합니다 `langchain-upstage`.
```
pip install -U langchain-upstage
```

다음 환경 변수를 설정해야 합니다.

`UPSTAGE_API_KEY`: Upstage 콘솔 의 Upstage API 키입니다 .

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`base_url`: 적용 가능한 모델 url

- 링크: https://developers.upstage.ai/docs/apis/chat


## Create an Upstage LLM

In [2]:
from langchain_upstage import ChatUpstage
# from openai import OpenAI

# Fetch the API key from environment variables, 
# .env 파일에 설정한 UPSTAGE_API_KEY 가져옴
upstage_api_key = os.getenv('UPSTAGE_API_KEY')


# 객체 생성
# llm = ChatUpstage()
llm = ChatUpstage(
    api_key=upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)
# client = OpenAI(
#     api_key=upstage_api_key,
#     base_url="https://api.upstage.ai/v1/solar"
# )


# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
print(f"[답변]: {llm.predict(question)}")

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[답변]: 서울.


## 프롬프트 템플릿의 활용


`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.
- 사용법: 리스트 형식으로 변수 이름을 정의합니다.


In [3]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain 객체


`LLMChain`

- LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다
- 사용법
  - `prompt`: 앞서 정의한 PromptTemplate 객체를 사용합니다.
  - `llm`: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.


In [4]:
from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### run()


`run()` 함수로 템플릿 프롬프트 실행


In [5]:
# 체인 실행: run()
llm_chain.run(country="일본")

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'일본의 수도는 도쿄(Tokyo)입니다.'

In [6]:
# 체인 실행: run()
llm_chain.run(country="캐나다")

'캐나다의 수도는 오타와(Ottawa)입니다.'

`predict()` 함수로 `run()` 함수를 대체할 수 있습니다.


In [7]:
llm_chain.predict(country="대한민국")

'서울특별시가 대한민국의 수도입니다.'

### apply()

`apply()` 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.


In [8]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

llm_chain.apply(input_list)

[{'text': '호주의 수도는 캔버라(Canberra)입니다.'},
 {'text': "중국(People's Republic of China)의 수도는 베이징(Beijing)입니다."},
 {'text': '네덜란드의 수도는 암스테르담이 아니라 헤이그(덴하그)입니다.'}]

`text` 키 값으로 결과 뭉치가 반환되었음을 확인할 수 있습니다.

이를 반복문으로 출력한다면 다음과 같습니다.


In [9]:
# input_list 에 대한 결과 반환
result = llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

호주의 수도는 캔버라(Canberra)입니다.
중국의 수도는 베이징(北京)입니다.
암스테르담


### `generate()`


`generate()` 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사합니다.

LLMResult는 토큰 사용량과 종료 이유와 같은 유용한 생성 정보를 자주 포함하고 있습니다.


In [10]:
# 입력값
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='호주의 수도는 캔버라(Canberra)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라(Canberra)입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 17, 'total_tokens': 32}, 'model_name': 'solar-1-mini-chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-464b9fa2-5cff-4d9f-8bb4-74bb31c600b9-0', usage_metadata={'input_tokens': 17, 'output_tokens': 15, 'total_tokens': 32}))], [ChatGeneration(text='베이징', generation_info={'finish_reason': 'eos', 'logprobs': None}, message=AIMessage(content='베이징', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 17, 'total_tokens': 20}, 'model_name': 'solar-1-mini-chat', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-198cc451-e33a-4434-b684-ba592029d56d-0', usage_metadata={'input_tokens': 17, 'output_tokens': 3, 'total_tokens': 20}))], [ChatGeneration(text='암스테

In [11]:
# 답변 출력
generated_result.generations

[[ChatGeneration(text='호주의 수도는 캔버라(Canberra)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라(Canberra)입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 17, 'total_tokens': 32}, 'model_name': 'solar-1-mini-chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-464b9fa2-5cff-4d9f-8bb4-74bb31c600b9-0', usage_metadata={'input_tokens': 17, 'output_tokens': 15, 'total_tokens': 32}))],
 [ChatGeneration(text='베이징', generation_info={'finish_reason': 'eos', 'logprobs': None}, message=AIMessage(content='베이징', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 17, 'total_tokens': 20}, 'model_name': 'solar-1-mini-chat', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-198cc451-e33a-4434-b684-ba592029d56d-0', usage_metadata={'input_tokens': 17, 'output_tokens': 3, 'total_tokens': 20}))],
 [ChatGeneration(text='암스테르담', gener

In [12]:
# 토큰 사용량 출력
generated_result.llm_output

{'token_usage': {'completion_tokens': 23,
  'prompt_tokens': 53,
  'total_tokens': 76},
 'model_name': 'solar-1-mini-chat'}

In [13]:
# run ID 출력
generated_result.run

[RunInfo(run_id=UUID('464b9fa2-5cff-4d9f-8bb4-74bb31c600b9')),
 RunInfo(run_id=UUID('198cc451-e33a-4434-b684-ba592029d56d')),
 RunInfo(run_id=UUID('89f65273-4982-4606-ab4b-3a093317764d'))]

In [14]:
# 답변 출력
for gen in generated_result.generations:
    print(gen[0].text.strip())

호주의 수도는 캔버라(Canberra)입니다.
베이징
암스테르담


### 2개 이상의 변수를 템플릿 안에 정의


2개 이상의 변수를 적용하여 템플릿을 생성할 수 있습니다.

이번에는 2개 이상의 변수(`input_variables`) 를 활용하여 템플릿 구성을 해보겠습니다.


In [15]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇시간이야?')

In [16]:
# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [17]:
# 체인 실행: run()
print(llm_chain.run(area1="서울", area2="파리"))

7시간입니다.


In [18]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

파리는 그리니치 표준시(GMT)로부터 1시간 빠른 중앙유럽 표준시(CET)에 해당합니다. 뉴욕은 그리니치 표준시(GMT)로부터 5시간 빠른 동부 표준시(EST)에 해당합니다. 따라서 파리에서 뉴욕으로 갈 때 시차를 고려하면, 뉴욕은 파리보다 4시간 빠릅니다.
서울 와 하와이 의 시차는 17시간입니다.
캔버라와 베이징의 시차는 1시간입니다.


## 스트리밍(streaming)

스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용합니다.

다음과 같이 `streaming=True` 로 설정하고 스트리밍으로 답변을 받기 위한 `StreamingStdOutCallbackHandler()` 을 콜백으로 지정합니다.


In [23]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatUpstage(
    temperature=1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    api_key=upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [24]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.predict(question)

대한민국, 또는 South Korea는 동아시아의 한반도에 위치한 나라입니다. 한반도는 북쪽으로 북한, 남쪽으로 서해와 동해로 둘러싸여 있습니다. 대한민국은 면적 약 100,313 평방킬로미터(35,491 평방마일), 인구는 약 5,174만 명에 이르는 나라로, 서울에는 약 974만 명이 거주하고 있습니다. 언어는 한국어, 국기는 태극기, 화폐는 한국 원(한국 화폐단위)을 사용합니다. 

대한민국은 현대적인 시설과 인프라를 갖춘 나라로 잘 알려져 있습니다. 서울은 특히 인구 밀도가 높고, 경제적으로 발달한 도시입니다. 한국과 일본은 문화적, 역사적으로 깊은 관계를 가지고 있으며, 양국은 현대사에서 어려움을 겪기도 했습니다.

대한민국 국민은 예의 바르고 친절하며, 예의범절을 중시하는 사회로 알려져 있습니다. 많은 한국인들은 가족 중심적이며, '조상 숭배'라고 불리는 조상들과의 깊은 연결과 존경심을 가지고 있습니다. 한국 문화는 한식, K-pop 음악, 드라마, 영화, 전통 예술 등 다양한 분야에서 세계적으로 인기를 얻고 있습니다.

대한민국 사회는 경제적인 격차가 상당합니다. 한국은 경제적으로 부유한 나라로 여겨지지만, 빈부 격차가 크다는 문제가 있습니다. 그러나 전반적으로 교육 수준이 높고, 시민 의식이 높으며, 정치적인 자유와 민주주의가 보장되어 있습니다.

현대 한국 역사는 일본에서 독립한 이후부터 현재까지 이어져 왔습니다. 한국은 전쟁, 권위주의 정권, 인권 침해 등 다양한 어려움을 겪었지만, 경제적인 발전과 민주주의의 성장을 이뤄냈습니다. 오늘날 한국은 세계적으로 영향력 있는 나라로서 국제 사회에 기여하고 있습니다.

한국 여행은 안전하고 풍부한 경험을 제공하는 나라입니다. 세계적으로 유명한 K-pop 콘서트, 드라마 촬영지, 한강의 아름다운 야경, 역사적인 유적지 등 다양한 관광 명소가 있습니다. 한국은 현대적인 도시와 전통적인 문화가 조화롭게 어우러진 매력적인 나라로, 다양한 매력을 즐길 수 있습니다.